# Dataset Profiling

Este notebook sirve para realizar el profiling del dataset y entender su estructura.

Incluye análisis exploratorio de datos y generación de estadísticas.

In [ ]:
import json
import pandas as pd
from pathlib import Path

In [ ]:
# Ruta al dataset
file_path = "farmers-protest-tweets-2021-2-4.json"

In [ ]:
# TODO: Cargar y analizar el dataset
# - Número de tweets
# - Estructura de cada tweet
# - Campos disponibles
# - Distribución de fechas
# - Estadísticas básicas